# Домашнее задание 1. 

-  "Оптимальность" наивного Байесовского классификатора
- Оценка "важности" признаков (feature importance)

Для тех кто не знаком с библиотекой pandas я рекомендую вот эти два практических туториала:

-  https://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/pandas.ipynb (1-3 часа на прохождение, 30 вопросов с видео разборами)
- https://github.com/guipsamora/pandas_exercises/ 11 топиков покрывающих весь основной функционал библиотеки


## 1. Наивный Байес

Ваша задача сравнить вероятности получаемые наивным байесовским классификатором с вероятностями получаемыми если не делать предположения о независимости признаков. Мы будем делать это для 2 признаков рассмотренных на семинаре: Пол и Класс билета (пассажира).

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('titanic_data.csv')

In [4]:
X = pd.DataFrame()

X['Sex'] = df['Sex'].map({
    'Male':0,
    'Female':1
})
X['PassengerClass'] = df['Passenger Class'].map({
    'First': 1,
    'Second': 2,
    'Third': 3
})

X['Survived'] = df['Survived'].map({
    'Yes': 1,
    'No': 0
})

y = X['Survived']
X = X.drop(['Survived'], axis=1)

In [5]:
print(y)

0       1
1       1
2       0
3       0
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: Survived, Length: 1309, dtype: int64


## 1.1 Разделим данные на тренировочную и тестовую части

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=5)

## 1.2 Реализуйте функцию `get_probability_survived_bayes`

Которая считает честные вероятности выживания при условии заданного класса билета и пола:

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{Passenger Class}, \text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}|\text{Passenger Class}) p(\text{Passenger Class})}
$$

## Помните что оценка вероятностей всегда должна происходить на тренировчной выборке.

In [8]:
from typing import Tuple

In [9]:
# Hint 1. Используйте pandas.groupby
# Hint 2. Вы возвращаете tuple p_not_survived, p_survived, в котором
# вероятность выжить находится на второй позиции, чтобы когда вы берете аргмакс получать метки классов 0/1 (не выжил/выжил)

def get_probability_survived_bayes(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns true probabilities tuple(p_not_survived, p_survived)'''
    data = X_train.copy()
    data['Survived'] = y_train

    surv_gr = data.groupby('Survived').get_group(1)
    no_surv_gr = data.groupby('Survived').get_group(0)

    p_surv = len(surv_gr) / len(data)
    p_no_surv = len(no_surv_gr) / len(data)

    cl_surv_gr = surv_gr.groupby('PassengerClass').get_group(pclass)
    cl_no_surv_gr = no_surv_gr.groupby('PassengerClass').get_group(pclass)
    
    p_cl_surv = len(cl_surv_gr) / len(surv_gr)
    p_cl_no_surv = len(cl_no_surv_gr) / len(no_surv_gr)
    
    s_cl_surv_gr = cl_surv_gr.groupby('Sex').get_group(sex)
    s_cl_no_surv_gr = cl_no_surv_gr.groupby('Sex').get_group(sex)
    
    p_s_cl_surv = len(s_cl_surv_gr) / len(cl_surv_gr)
    p_s_cl_no_surv = len(s_cl_no_surv_gr) / len(cl_no_surv_gr)
    
    cl_gr = data.groupby('PassengerClass').get_group(pclass)
    p_cl = len(cl_gr)/len(data)
    
    s_cl_gr = cl_gr.groupby('Sex').get_group(sex)
    p_s_cl = len(s_cl_gr)/len(cl_gr)
    
    P_surv = p_surv * p_cl_surv * p_s_cl_surv / p_cl / p_s_cl
    P_no_surv = p_no_surv * p_cl_no_surv * p_s_cl_no_surv / p_cl / p_s_cl
    
    return P_no_surv, P_surv


In [10]:
print(get_probability_survived_bayes(1, 1))

(0.03960396039603961, 0.9603960396039604)


## 1.3 Реализуйте функцию `get_probability_survived_bayes_naive`

Которая оценивает вероятность выжить в предположении о независимости признаков

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}) p(\text{Passenger Class})}
$$

In [11]:
def get_probability_survived_bayes_naive(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns tuple(p_not_survived, p_survived), 
    under naive assumption of features independence'''
    
    data = X_train.copy()
    data['Survived'] = y_train

    surv_gr = data.groupby('Survived').get_group(1)
    no_surv_gr = data.groupby('Survived').get_group(0)

    p_surv = len(surv_gr) / len(data)
    p_no_surv = len(no_surv_gr) / len(data)

    cl_surv_gr = surv_gr.groupby('PassengerClass').get_group(pclass)
    cl_no_surv_gr = no_surv_gr.groupby('PassengerClass').get_group(pclass)
    
    p_cl_surv = len(cl_surv_gr) / len(surv_gr)
    p_cl_no_surv = len(cl_no_surv_gr) / len(no_surv_gr)
    
    s_surv_gr = surv_gr.groupby('Sex').get_group(sex)
    s_no_surv_gr = no_surv_gr.groupby('Sex').get_group(sex)
    
    p_s_surv = len(s_surv_gr) / len(surv_gr)
    p_s_no_surv = len(s_no_surv_gr) / len(no_surv_gr)
    
    cl_gr = data.groupby('PassengerClass').get_group(pclass)
    p_cl = len(cl_gr)/len(data)
    
    s_gr = data.groupby('Sex').get_group(sex)
    p_s = len(s_gr)/len(data)
    
    P_surv = p_surv * p_cl_surv * p_s_surv / p_cl / p_s
    P_no_surv = p_no_surv * p_cl_no_surv * p_s_no_surv / p_cl / p_s
    
    norm = P_no_surv + P_surv
    
    return P_no_surv / norm, P_surv / norm
    
# Hint 1. Поскольку вы делаете предположении о независимости признаков, то p(y=1|...) + p(y=0| ...) некорректно отнормированы
# и не будут суммироваться в 1. Не забудьте выполнить необходимую нормировку.

In [12]:
print(get_probability_survived_bayes_naive(1, 1))

(0.1109334203281885, 0.8890665796718115)


## 1.4 Используйте ваши функции для того чтобы сделать предсказание на тесте


1. Для всех 6 уникальных комбинаций значений признаков сравните вероятности принадлежности классу 1
 - насколько они отличаются?
 - влияют ли эти отличия на предсказание класса объекта, при пороге $p = 0.5$?
2. Посчитайте точность (accuracy) на тренировочной и тестовой выборке для двух подходов, какой подход дает более высокую точность? (используйте порог $p = 0.5$).
3. С вашей точки зрения в данной задаче следует максимизировать Recall или Precision? Объясните свой ответ.
4. В зависимости от вашего ответа на предыдущий вопрос, какой классификатор и какой порог вероятности необходимо выбрать. Сравните матрицу ошибок (confusion matrix) для найденного порога и порога $p = 0.5$ и прокомментируйте отличия.


In [13]:
from tqdm.notebook import tqdm

In [14]:
y_pred_proba_test_naive = []
y_pred_proba_train_naive = []
y_pred_proba_test_bayes = []
y_pred_test_naive = []
y_pred_train_naive = []

for _, (sex, pclass) in X_test.iterrows():
    y_pred_proba_test_naive.append(get_probability_survived_bayes_naive(pclass, sex))
    y_pred_proba_test_bayes.append(get_probability_survived_bayes(pclass, sex))

for _, (sex, pclass) in X_train.iterrows():
    y_pred_proba_train_naive.append(get_probability_survived_bayes_naive(pclass, sex))
    
#train set prediction
def get_prediction(p, y_pred_proba, X, y_pred):
    for _, (sex, pclass) in X.iterrows():
        if(y_pred_proba[len(y_pred)][0] > p):
            #not survived
            y_pred.append(0)
        else:
            #survived
            y_pred.append(1)

            
get_prediction(0.5, y_pred_proba_test_naive, X_test, y_pred_test_naive)
get_prediction(0.5, y_pred_proba_train_naive, X_train, y_pred_train_naive)  

In [15]:
naive_b_proba = []
b_proba = []

for pclass in range (1, 4):
    b_proba.append(get_probability_survived_bayes(pclass, 0))
    naive_b_proba.append(get_probability_survived_bayes_naive(pclass, 0))

for pclass in range (1, 4):
    b_proba.append(get_probability_survived_bayes(pclass, 1))
    naive_b_proba.append(get_probability_survived_bayes_naive(pclass, 1))

print(naive_b_proba)
print(b_proba)
#при пороге 0.5 разницы в предсказании между naive_b_proba и b_proba не будет

[(0.6123888566138307, 0.3876111433861693), (0.8069863930278662, 0.19301360697213374), (0.8787934903367802, 0.12120650966321975), (0.1109334203281885, 0.8890665796718115), (0.2482326217955581, 0.7517673782044418), (0.3641139278258808, 0.6358860721741193)]
[(0.6363636363636365, 0.36363636363636365), (0.8738738738738738, 0.12612612612612611), (0.8523076923076921, 0.1476923076923077), (0.03960396039603961, 0.9603960396039604), (0.14754098360655735, 0.8524590163934426), (0.45112781954887216, 0.5488721804511278)]


In [16]:
#2
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
#calculate accuracy
test_accure = accuracy_score(y_test, y_pred_test_naive)
test_recall = recall_score(y_test, y_pred_test_naive)
test_precision = precision_score(y_test, y_pred_test_naive)
print("test set accuracy:" ,test_accure)

train_accure = accuracy_score(y_train, y_pred_train_naive)
print("train set accuracy:", train_accure)

#на тренировочной выборке точность предсказания выше

print("test recall:", test_recall, "test precision:", test_precision)

test set accuracy: 0.7645739910313901
train set accuracy: 0.7879490150637312
test recall: 0.6964285714285714 test precision: 0.6842105263157895


3) Мне кажется следует максимизировать метрику precision, так как в подобных задачах мы хотим предсказать наихудший возможный исход по количеству жертв (даже если на самом деле некоторые из них не погибнут). 

In [21]:
#4
max_prec_naive = 0
p_prob_naive = 0
max_prec_bayes = 0
p_prob_bayes = 0

for i in range (1, 1000):
    y_pred_naive = []
    y_pred_bayes = []
    
    p = (1000 - i) / 1000
    
    get_prediction(p, y_pred_proba_test_naive, X_test, y_pred_naive)
    get_prediction(p, y_pred_proba_test_bayes, X_test, y_pred_bayes)
    
    prec_naive = precision_score(y_test, y_pred_naive)
    prec_bayes = precision_score(y_test, y_pred_bayes)

    if(prec_naive > max_prec_naive):
        max_prec_naive = prec_naive
        p_prob_naive = p
    
    if(prec_bayes > max_prec_bayes):
        max_prec_bayes = prec_bayes
        p_prob_bayes = p

print("max_prec_naive:", max_prec_naive, p_prob_naive)
print("max_prec_bayes:", max_prec_bayes, p_prob_bayes)

y_pred_naive = []
y_pred_bayes = []
get_prediction(p_prob_naive, y_pred_proba_test_naive, X_test, y_pred_naive)
get_prediction(p_prob_bayes, y_pred_proba_test_bayes, X_test, y_pred_bayes)

accure_bayes_p = accuracy_score(y_test, y_pred_naive)
accure_naive_p = accuracy_score(y_test, y_pred_bayes)

print("accuracy for bayes, p =", p_prob_bayes, accure_bayes_p)
print("accuracy for naive, p =", y_pred_naive, accure_naive_p)

/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: Un

/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: Un

/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: Un

/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: Un

/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ekatrina/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: Un

max_prec_naive: 0.9767441860465116 0.248
max_prec_bayes: 0.9767441860465116 0.147
accuracy for bayes, p = 0.147 0.7152466367713004
accuracy for naive, p = 0.147 0.7152466367713004


In [27]:
#confusion matrix
con_mat_o_5 = confusion_matrix(y_test, y_pred_test_naive)
con_mat_p = confusion_matrix(y_test, y_pred_naive)

print(con_mat_o_5)
print(con_mat_p)

[[224  54]
 [ 51 117]]
[[277   1]
 [126  42]]


## 1.5 Обучите имплементацию `MultinomialNB` из `sklearn`

1. Тот же вопрос что и 1.4.1, прокомментируйте отличия в оценке вероятности ваших имлементаций и имплементации
`MultinomialNB`. Влияют ли они на предсказание классов при стандартном пороге вероятности? Заполните таблицу `probabilities`.

In [28]:
from sklearn.naive_bayes import MultinomialNB

probabilities = pd.DataFrame()
probabilities['Sex'] = [0,0,0,1,1,1]
probabilities['PassengerClass'] = [1,2,3,1,2,3]
probabilities['Bayes'] = [0]*6
probabilities['BayesNaive'] = [0]*6
probabilities['MultinomialNB'] = [0]*6

X_mini_test = pd.DataFrame()
X_mini_test['Sex'] = probabilities['Sex']
X_mini_test['PassengerClass'] = probabilities['PassengerClass']


clf = MultinomialNB()
clf.fit(X_train, y_train)

print(X_train)

      Sex  PassengerClass
615     0               3
287     0               1
720     0               3
222     0               1
244     0               1
...   ...             ...
73      1               1
1142    0               3
998     0               3
206     0               1
867     1               3

[863 rows x 2 columns]


In [29]:
probabilities['MultinomialNB'] = clf.predict_proba(X_mini_test)

for i in range (0, 6):
    probabilities['Bayes'][i] = b_proba[i][0]
    probabilities['BayesNaive'][i] = naive_b_proba[i][0]

print(probabilities)

y_pred_test_model = clf.predict(X_test)
accure_model1 = accuracy_score(y_test, y_pred_test_model)
print(accure_model1)

   Sex  PassengerClass     Bayes  BayesNaive  MultinomialNB
0    0               1  0.636364    0.612389       0.671130
1    0               2  0.873874    0.806986       0.722516
2    0               3  0.852308    0.878793       0.768640
3    1               1  0.039604    0.110933       0.296836
4    1               2  0.147541    0.248233       0.350069
5    1               3  0.451128    0.364114       0.407318
0.7645739910313901


/tmp/ipykernel_4493/2170414471.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probabilities['Bayes'][i] = b_proba[i][0]
/tmp/ipykernel_4493/2170414471.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probabilities['BayesNaive'][i] = naive_b_proba[i][0]


## 2.1 Используя MultinomialNB обучите классификатор на бОльшем количестве признаков

1. Подготовьте признаки 
- 'Passenger Class',
- 'Sex',
- 'Age',
- 'No of Siblings or Spouses on Board',
- 'No of Parents or Children on Board',
- 'Port of Embarkation'

для обучения:
- закодируйте столбец `Port of Embarkation` по аналогии с `Passenger Class`
- заполните пропуски в столбце `Age` средним возрастом в датасете используйте `pd.DataFrame.fillna(...)`
- заполните пропуске в столбце `Port of Embarkation` самым частым портом посадки
- для разбиения на тренировчную и тестовую часть данных используйте `train_size=0.66` и `random_state=5`

2. обучите MultinomialNB модель на тренировочной части данных
2. Как изменилась точность вашей модели на тестовой части данных по сравнению с моделью обученной на 2 признаках?
3. Что означает признак `Life Boat`? Следуют ли на ваш взгляд включать его в список признаков для обучения модели? 
- Опишите сценарий (использования модели) в котором его следует включать в список признаков. 
- Опишите сценарий (использования модели) в котором его не следует включать в список признаков.
5. Какая ошибка содержится в предложенной последовательности действий в п. 2.1? 

In [30]:
mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(mean)

often = df['Port of Embarkation'].mode()[0]
print(often)
df['Port of Embarkation'] = df['Port of Embarkation'].fillna(often)

X['Port_of_Embarkation'] = df['Port of Embarkation'].map({
    'Southampton': 1,
    'Cherbourg': 2,
    'Queenstown': 3
})

X['Age'] = df['Age']
X['No of Siblings or Spouses on Board'] = df['No of Siblings or Spouses on Board']
X['No of Parents or Children on Board'] = df['No of Parents or Children on Board']

Southampton


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=5)

clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [32]:
y_predict = []
y_predict = clf.predict(X_test)

accure_model2 = accuracy_score(y_test, y_predict)
print(accure_model2)

0.7668161434977578


## 2.2. Оценка важности признаков

Мы воспользуемся простым (и не очень хорошим^[1]) методом оценки важности признаков:

1. Воспользуйтесь моделью из п.2. Качество модели на тестовой части данных это ваш бейзлайн
2. Исключая по 1 признаку из `X` обучайте модель `MultinomialNB` и измеряйте ее точность на тестовой части данных
3. Сохраняйте величину падения или роста качества по сравнению с бейзлайном. Если точность упала значит признак "полезный", если выросло - "вредный". Абсолютная величина падения или роста в данном случае не так важна, важен только порядок признаков. Такая оценка называется качественной (в противовес количественной).
4. Прокомментируйте полученный результат, насколько он согласуется с вашей интуицией?
5. В чем на ваш взляд могут быть преимущества использования мЕньшего числа признаков при обучении моделей? (не обязательно в контексте данной задачи)


[1] а впрочем какие из них хороши?

In [112]:
def learn(xtrain, ytrain, xtest, ytest):
    lclf = MultinomialNB()
    lclf.fit(xtrain, ytrain)
    
    predict = lclf.predict(xtest)
    accure = accuracy_score(ytest, predict)
    
    return accure

def del_priznak(x_train, x_test, ytrain, ytest):
    for i in x_train:
        new_x_train = x_train.drop(columns = i)
        new_x_test = x_test.drop(columns = i)
        
        accure = learn(new_x_train, ytrain, new_x_test, ytest)
        print(i, ": ", accure)
    
del_priznak(X_train, X_test, y_train, y_test)        

Sex :  0.6569506726457399
PassengerClass :  0.7443946188340808
Port_of_Embarkation :  0.7623318385650224
Age :  0.7690582959641256
No of Siblings or Spouses on Board :  0.7668161434977578
No of Parents or Children on Board :  0.7645739910313901


## 3. Подготовьте репорт о выполненном задании

Подготовьте текстовый репорт о выполенном домашнем задании. Ваш репорт должен включать ответы на вопросы:
- 1.4.1-1.4.4
- 1.5.1
- 2.1.3-2.1.5
- 2.2.3-2.2.5

Вы можете вставлять в текст таблицы и/или графики на ваше усмотрение (для п. 1.5.1 обязательно вставьте таблицу или скриншот таблицы probabi), текст репорта должен быть в двухколоночном формате не более 2 страниц 12 кеглем Times New Roman (**крайне** желательно 1 страница), см шаблон https://docs.google.com/document/d/1YqnO0OFbTR7XgzmdWryQF22J1kl9Fg7hSlNbqa6Jbiw/edit?usp=sharing

Подготовленный репорт и jupyter notebook c вашим решением необходимо отправить на почту kurmukovai@gmail.com, с темой письма `[iitp-intro-ds-2023-ha1-Surname]`, например `[iitp-intro-ds-2023-Kurmukov]` до 23:59:59 МСК 25.02.2023 .